<a href="https://colab.research.google.com/github/TommyChoiDS/ds-section1-project/blob/main/AI_05_%E1%84%8E%E1%85%AC%E1%84%8C%E1%85%AE%E1%86%BC%E1%84%92%E1%85%AE%E1%86%AB_Section2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 선정 이유 및 문제 정의

- 데이터 설명 : 프로젝트에 활용된 데이터는 미국 Iowa주 ames의 집값 데이터이다. 자주 인용되는 Boston 집값 데이터에 비해 다양한 변수가 있다. 

- 데이터 선정 이유 : ames 집값 예측데이터는 kaggle getting started 대회에서도 사용되고 있으며, 변수의 수가 많아 다양한 Feature Engineering을 시도해 볼 수 있다.  Section2에서 배운 내용을 잘 활용하여 좋은 모델을 만들 수 있을것으로 생각해 선정하게 되었다.


- 주어진 문제는 ames의 집값을 예측하는것이고, 집값은 연속형 실수 데이터이기 때문에 회귀유형의 문제로 정의하고 집값을 예측한다.


## 라이브러리 설치 및 불러오기

In [ ]:
# !pip install category_encoders
# !pip install catboost
# !pip install -U pandas-profiling
# !pip install lightgbm
# !pip install eli5

In [ ]:
from google.colab import drive
drive.mount('./gdrive')
!mkdir /root/.kaggle/
!cp /content/gdrive/MyDrive/kaggle.json /root/.kaggle/   
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c home-data-for-ml-course

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
import sklearn.metrics
from sklearn.preprocessing import LabelEncoder
from category_encoders import OrdinalEncoder

## 2. 데이터 불러오기

In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train, valid = train_test_split(train, train_size = 0.8, test_size = 0.2, random_state = 2)
sub = pd.read_csv('sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train['PavedDrive'].value_counts()

## 변수 설명



*  MSSubClass : 건물 등급
*  MSZoning : 일반 구역 분류
*  LotFrontage : 부동산에 연결된 거리의 선형 피트
*  LotArea : 평방 피트 단위의 부지 크기
*  Street : 도로 접근 유형
*  Alley : 골목 접근 유형
*  LotShape : 부동산의 일반적인 모양
*  LandContour : 부동산의 평탄도
*  Utilities : 사용 가능한 유틸리티 유형(에너지 유형이란 뜻, 전기, 가스, 수도 등을 유틸리티로 통칭함)
*  LotConfig : 로트 구성
*  LandSlope : 재산의 경사
*  Neighborhood : 에임스시 경계 내의 물리적 위치 
*  Condition1 : 주요 도로 또는 철도와의 근접성
*  Condition2 : 주요 도로 또는 철도와의 근접성 (초가있는 경우)
*  BldgType : 주거 유형
*  HouseStyle : 주거 스타일
*  OverallQual : 전체 재료 및 마감 품질
*  OverallCond : 전체 상태 등급
*  YearBuilt : 원래 건설 날짜
*  YearRemodAdd : 리모델링 날짜
*  RoofStyle : 지붕 유형
*  RoofMatl : 지붕 재료
*  Exterior1st : 집 외장재1
*  Exterior2nd : 집 외장재2 (하나 이상의 재료 인 경우)
*  MasVnrType : 벽돌 베니어 유형
*  MasVnrArea : 벽돌 베니어 면적 (평방 피트)
*  ExterQual : 외장재 품질
*  ExterCond : 외장재의 현황
*  Basement : 지하실 유무
*  BsmtQual : 지하실 높이
*  BsmtCond : 지하실의 일반 상태
*  BsmtExposure : 워크 아웃 또는 정원 수준의 지하 벽
*  BsmtFinType1 : 지하실 마감 면적의 품질
*  BsmtFinSF1 : 유형 1 완성 평방 피트
*  BsmtFinType2 : 두 번째 완성 된 영역의 품질 (있는 경우)
*  BsmtFinSF2 : 유형 2 마감 평방 피트
*  BsmtUnfSF : 미완성 된 지하실 면적
*  TotalBsmtSF : 지하 총 평방 피트
*  Heating : 난방 유형
*  HeatingQC : 난방 품질 및 상태
*  CentralAir : 중앙 에어컨
*  Electrical : 전기 시스템
*  1stFlrSF : 1 층 평방 피트
*  2ndFlrSF : 2 층 평방 피트
*  LowQualFinSF : 저품질 마감 평방 피트 (모든 층)
*  GrLivArea : 지상 (지상) 거실 면적 평방 피트
*  BsmtFullBath : 지하 Fullbath 여부
*  BsmtHalfBath : 지하 Halfbath 여부
*  FullBath : Fullbath 여부(Fullbath : 세면대1, 욕조1, 샤워부스1, 변기1로 구성된 화장실)
*  HalfBath : Halfbath 여부(Halfbath : 세면대1, 변기1로 구성된 화장실)
*  Bedroom : 지하층 이상의 침실 수
*  Kitchen : 주방 수
*  KitchenQual : 주방 품질
*  TotRmsAbvGrd : 기준 이상의 방 갯수 (화장실 제외)
*  Functional : 홈 기능 등급
*  Fireplaces : 벽난로 수
*  FireplaceQu : 벽난로 품질
*  GarageType : 차고 위치
*  GarageYrBlt : 차고 건설 연도
*  GarageFinish : 차고 내부 마감
*  GarageCars : 차량 수용 가능 차고 크기
*  GarageArea : 차고 크기 (평방 피트)
*  GarageQual : 차고 품질
*  GarageCond : 차고 상태
*  PavedDrive : 포장 된 진입로
*  WoodDeckSF : 목재 데크 면적 (평방 피트)
*  OpenPorchSF : 평방 피트 단위의 열린 현관 영역
*  EnclosedPorch : 닫힌 현관 영역 (평방 피트)
*  3SsnPorch : 평방 피트 단위의 3 계절 현관 면적
*  ScreenPorch : 스크린 현관 영역 (평방 피트)
*  PoolArea : 수영장 면적 (평방 피트)
*  PoolQC : 수영장 품질
*  Fence : 울타리 품질
*  MiscFeature : 다른 카테고리에서 다루지 않는 기타 기능
*  MiscVal : 기타 기능의 $ Value
*  MoSold : 판매 월
*  YrSold : 판매 연도
*  SaleType : 판매 유형
*  SaleCondition : 판매 조건

# 2. 데이터를 이용한 가설 및 평가지표, 베이스라인 선택

- 가설:
    - 건물이 얼마나 오래되었는지에 따라 집의 가격이 달라질 것이다.
    - 리모델링한지 얼마나 오래되었는지에 따라 집의 가격이 달라질 것이다.
    - 건물의 총 면적에 따라 집의 가격이 달라질 것이다.
- 예측하고자 하는 변수
    -  target: SalePrice($)
- 베이스라인 모델(해당 베이스라인 모델과 평가지표를 선택한 이유를 설명)
  -  앙상블 모델의 기본이라 할 수 있는 RandomForest를 baseline model로 지정한다.
  -  RandomForest의 hyperparameter를 기본값으로 두고 학습을 진행하여 성능향상의 기준으로 정한다.
- 평가지표 선택
    - 회귀문제 평가지표:
    - RMSLE (Root Mean Squared Log Error): 큰 오차값에 대해 영향을 덜 받는 RMSLE(Root-Mean-Squared-Log-Error)를 사용.


# 3. EDA/데이터 전처리

##  EDA

In [ ]:
# data shape
print('train:',train.shape)
print('valid:',valid.shape)
print('test:',test.shape)

In [ ]:
# data profiling
profile = ProfileReport(train, minimal = True).to_notebook_iframe()



## 어떤 변수가 target과 상관관계가 높은지 확인
* target과 상관관계가 높은것부터 내림차순으로 나열해본 결과는 다음과 같다.
```
OverallQual      0.784354
GrLivArea        0.686425
GarageCars       0.632043
GarageArea       0.621604
TotalBsmtSF      0.611302
```

In [ ]:
train.corr()["SalePrice"].sort_values(ascending = False)

## 상관계수가 높은 변수들의 산점도를 그려 target변수와 어떤 관계가 있는지 파악

In [ ]:
plt.figure(figsize = (15,12));

plt.subplot(321);
sns.scatterplot(train["OverallQual"],train["SalePrice"]);

plt.subplot(322);
sns.scatterplot(train["GrLivArea"],train["SalePrice"]);

plt.subplot(323);
sns.scatterplot(train["GarageCars"],train["SalePrice"]);

plt.subplot(324);
sns.scatterplot(train["GarageArea"],train["SalePrice"]);

plt.subplot(325);
sns.scatterplot(train["TotalBsmtSF"],train["SalePrice"]);

plt.subplot(326);
sns.scatterplot(train["1stFlrSF"],train["SalePrice"]);


## 로그변환

  * 선형 모델을 사용할 때, target변수의 분포가 왼쪽으로 치우친 모양일 경우 target변수에 로그변환을 한 후, 예측한 결과를 다시 지수변환을 해준다.
  * 로그변환이 필요한지 알아보기 위해 SalePrice의 분포를 확인한다.
  * 분포를 확인한 결과, SalePrice의 분포가 왼쪽으로 치우쳐져 있어 선형모델 학습시 로그 변환이 필요하다.
  * 사용할 모델은 부스팅계열 모델이고, 비선형 모델이므로 로그변환을 하지 않는다.

In [ ]:
plt.figure(figsize = (15,12))
sns.distplot(train['SalePrice'])

## 데이터 전처리


 - 이상치 처리 : 
    - 상관계수가 높은 변수들로 산점도를 그려본 결과, target변수와 선형관계를 보인다. 그러나 GrLivArea변수의 산점도에서 볼 수 있듯 거실 면적이 넓은데도 판매 가격이 높지 않은 경우가 있다. 이러한 데이터는 모델에 영향을 줄 수 있기 때문에 이상치로 판단하고 제거해준다.
    - 이상치를 제거하는 이유는 모델링에 영향을 주기 때문이므로, train 데이터셋의 이상치만 제거하고 validation과 test 데이터셋의 이상치는 제거하지 않는다.
 - 결측값 처리 : 
   - 수치형 변수의 결측치는 이상치에 영향을 덜 받는 중앙값으로 대체하고, 범주형 변수는 최빈값으로 대체한다.
 - 불필요한 컬럼 처리 :
    - ID 변수는 단순한 index이므로 모델 성능 향상에 의미가 없다고 판단하여 제거한다.
    - Utilities 변수의 NoSeWa 항목은 train데이터에서 단 1개, test데이터에서는 전혀 없으므로 의미가 없다고 판단하여 제거한다.
    - Street 변수의 Grvl 항목은 train데이터에서 4개, test데이터에서는 6개 이므로 의미가 없다고 판단하여 제거한다.
 - Feature Engineering :
    - 판매시점의 연도와 건물이 지어진 연도의 차이로 건물이 얼마나 오래되었는지 나타내는 변수를 새로 생성한다.
    - 판매시점의 연도와 건물이 리모델링된 연도의 차이로 건물을 리모델링한지 얼마나 오래되었는지 나타내는 변수를 새로 생성한다.
    - 지상 거실 면적과 지하 면적을 합하여 건물의 총 면적을 나타내는 변수를 새로 생성한다.

In [ ]:
train

In [ ]:
# preprocessing

def Feature_engineering(df):


  df['old'] = df['YrSold'] - df['YearBuilt'] # 판매시점에 건물이 얼마나 오래되었는지 나타내는 변수
  df['since_remod'] = df['YrSold'] - df['YearRemodAdd'] # 판매시점에 건물을 리모델링한지 얼마나 오래되었는지 나타내는 변수
  df['totalarea'] = df['GrLivArea'] + df['TotalBsmtSF'] # 건물의 총 면적을 나타내는 변수  

  # 결측치 대체
  df_col = df.loc[:,(df.dtypes == object)]
  df_col = df_col.fillna(df_col.mode().iloc[0])
  df_num = df.loc[:,(df.dtypes != object)]
  df_num = df_num.fillna(df_num.median())

  df = pd.concat([df_col,df_num],axis = 1)



  df = df.drop(['Utilities','Street'], axis = 1)


  return df

train = Feature_engineering(train)
valid = Feature_engineering(valid)
test = Feature_engineering(test)

# 이상치 제거
train = train.drop(train[(train["GrLivArea"] > 4000)&(train["SalePrice"] < 300000)].index)
train = train.drop(train[(train["GarageArea"] > 1300)&(train["SalePrice"] < 300000)].index)


train_x = train.drop('SalePrice', axis = 1)
valid_x = valid.drop('SalePrice', axis = 1)

train_y = train['SalePrice']
valid_y = valid['SalePrice']

## 이상치 제거 확인

In [ ]:
plt.figure(figsize = (15,12));

plt.subplot(321);
sns.scatterplot(train["OverallQual"],train["SalePrice"]);

plt.subplot(322);
sns.scatterplot(train["GrLivArea"],train["SalePrice"]);

plt.subplot(323);
sns.scatterplot(train["GarageCars"],train["SalePrice"]);

plt.subplot(324);
sns.scatterplot(train["GarageArea"],train["SalePrice"]);

plt.subplot(325);
sns.scatterplot(train["TotalBsmtSF"],train["SalePrice"]);

plt.subplot(326);
sns.scatterplot(train["1stFlrSF"],train["SalePrice"]);


# 4. 머신러닝 방식 적용 및 교차검증
모델을 학습, 평가지표를 계산 후 베이스라인과 비교 
-> 어느정도 성능이 나왔다면, 교차 검증 (CV)을 통해서 일반화될 가능성이 있는지 확인, 모델 성능을 개선하기 위한 다양한 방법을 적용(Hyperparameter tuning)


최소 2개 이상의 모델을 만들어서 validation 점수를 보고, 최종 모델의 test 점수를 보고

  - Q 모델을 학습한 후에 베이스라인보다 잘 나왔나요? 그렇지 않다면 그 이유는 무엇일까요?
  - Q 모델 성능 개선을 위해 어떤 방법을 적용했나요? 그 방법을 선택한 이유는 무엇인가요?
  - 최종 모델에 관해 설명하세요.

## 1) Baseline

In [ ]:
# Baseline prediction

pipe_base = make_pipeline(
    OrdinalEncoder(),
    RandomForestRegressor(random_state = 2)
    
)

pipe_base.fit(train_x, train_y)
result = pipe_base.predict(valid_x)

print('Baseline :',sklearn.metrics.mean_squared_log_error(result,valid_y)**0.5)

## 2) XGBoost

In [ ]:
# XGBoost prediction

pipe_xgb = make_pipeline(
    OrdinalEncoder(),
    XGBRegressor(verbosity = 0)
    
)

pipe_xgb.fit(train_x, train_y)
result = pipe_xgb.predict(valid_x)

print('XGBoost :',sklearn.metrics.mean_squared_log_error(result,valid_y)**0.5)

## 3) LGBM

In [ ]:
# LightGBM prediction

pipe_lgbm = make_pipeline(
    OrdinalEncoder(),
    LGBMRegressor()
    
)
pipe_lgbm.fit(train_x, train_y)
result = pipe_lgbm.predict(valid_x)

print('LGBM :',sklearn.metrics.mean_squared_log_error(result,valid_y)**0.5)

## 4) CatBoost

In [ ]:
# Catboost prediction

pipe_cat = make_pipeline(
    OrdinalEncoder(),
    CatBoostRegressor(verbose=0)
)

pipe_cat.fit(train_x, train_y)
result = pipe_cat.predict(valid_x)

print('catboost :',sklearn.metrics.mean_squared_log_error(result,valid_y)**0.5)

## 5) Hyper parameter tuning

In [ ]:
import catboost
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
train_x = ordinal_encoder.fit_transform(train_x)
valid_x = ordinal_encoder.fit_transform(valid_x)


model = CatBoostRegressor()


grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

randomized_search_result = model.randomized_search(grid,
                                                   X=train_x,
                                                   y=train_y,
                                                   plot=True)

In [ ]:
randomized_search_result

In [ ]:
# Catboost prediction - RandomizedSearchCV

pipe_cat = make_pipeline(
    # OrdinalEncoder(),
    CatBoostRegressor(verbose=0, learning_rate = 0.03, depth=6, l2_leaf_reg = 3)
)

pipe_cat.fit(train_x, train_y)
result = pipe_cat.predict(valid_x)

print('catboost :',sklearn.metrics.mean_squared_log_error(result,valid_y)**0.5)

In [ ]:
sub['SalePrice'] = result
sub
sub.to_csv("sub.csv",index = False)

# 5. 머신러닝 모델 해석
SHAP, PDP 등을 통해서 모델이 관측치를 어떤 특성을 활용했거나, 어떤 특성이 타겟에 영향을 끼쳤는지 등을 해석할 수 있습니다. PDP, SHAP을 활용하여 최종 모델을 설명합니다.

  - Q 모델이 관측치를 예측하기 위해서 어떤 특성을 활용했나요?
  - Q 어떤 특성이 있다면 모델의 예측에 도움이 될까요? 해당 특성은 어떻게 구할 수 있을까요?

In [ ]:
from sklearn.pipeline import Pipeline
pipe_final = Pipeline([
    ('preprocessing', make_pipeline(OrdinalEncoder())),
    ('cat', CatBoostRegressor(iterations = 4000, verbose = 1000, max_depth = 6)) 
])

In [ ]:
pipe_final.fit(train_x, train_y)
print('검증 정확도: ', pipe_final.score(valid_x, valid_y))

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import eli5
from eli5.sklearn import PermutationImportance

# permuter 정의
permuter = PermutationImportance(
    pipe_final.named_steps['cat'], # model
    scoring='neg_mean_squared_log_error', # metric
    n_iter=5, # 다른 random seed를 사용하여 5번 반복
    random_state = 2
)

# permuter 계산은 preprocessing 된 X_val을 사용합니다.
valid_x_transformed = pipe_final.named_steps['preprocessing'].transform(valid_x)

# 실제로 fit 의미보다는 스코어를 다시 계산하는 작업입니다
permuter.fit(valid_x_transformed, valid_y);

In [ ]:
feature_names = valid_x.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values()

In [ ]:
# 특성별 score 확인
eli5.show_weights(
    permuter, 
    top=None, # top n 지정 가능, None 일 경우 모든 특성 
    feature_names=feature_names # list 형식으로 넣어야 합니다
)